Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

24/04/26 01:24:19 WARN Utils: Your hostname, Chanchals-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.28.17.96 instead (on interface en0)
24/04/26 01:24:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 01:24:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/26 01:24:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/26 01:24:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/26 01:24:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/26 01:24:20 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/04/26 01:24:20 WARN Utils: Service 'SparkUI' could not bind on port 4044. Atte

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
| Chanchal| 31|        10| 30000|
|   Khushi| 30|         8| 25000|
|   Kartik| 29|         4| 20000|
|     Ansh| 24|         3| 20000|
|Divyanshi| 21|         1| 15000|
|   Aniket| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
#[Age,Experience]----> new feature--->independent feature

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [9]:
output=featureassembler.transform(training)

In [10]:
output

DataFrame[Name: string, age: int, Experience: int, Salary: int, Independent Features: vector]

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
| Chanchal| 31|        10| 30000|         [31.0,10.0]|
|   Khushi| 30|         8| 25000|          [30.0,8.0]|
|   Kartik| 29|         4| 20000|          [29.0,4.0]|
|     Ansh| 24|         3| 20000|          [24.0,3.0]|
|Divyanshi| 21|         1| 15000|          [21.0,1.0]|
|   Aniket| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data=output.select("Independent Features","Salary")

In [14]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

24/04/26 16:22:48 WARN Instrumentation: [d87fe0d8] regParam is zero, which might cause numerical instability and overfitting.
24/04/26 16:22:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/26 16:22:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/04/26 16:22:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [16]:
regressor.coefficients

DenseVector([1771.9298, -298.2456])

In [17]:
### Intercepts
regressor.intercept

-21912.28070175451

In [18]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [19]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000|28280.701754385987|
|          [30.0,8.0]| 25000|28859.649122807026|
+--------------------+------+------------------+



In [20]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(6070.175438596507, 41733456.44813812)